In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
path = '/home/kunal/Downloads/lord_of_machines/'
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
#campaign = pd.read_csv(path +'campaign_data.csv')

In [ ]:
train_input = pd.read_csv(path + 'impact_encoded_train.csv')
test_input = pd.read_csv(path + 'impact_encoded_test.csv')

In [ ]:
user_features  = pd.read_csv(path + 'user_features.csv')

## Modelling part

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(40,random_state = 10)
user_features_matrix = pca.fit_transform(user_features.iloc[:,1:])

In [ ]:
user_features_matrix = pd.DataFrame(user_features_matrix)
user_features_matrix['user_id'] = user_features['user_id']
train_input = train_input.merge(user_features_matrix,on = 'user_id',how = 'left')


In [ ]:
#pca2 = PCA(30,random_state = 10)
#cluster_feature = pca2.fit_transform(campaign_features.iloc[:,1:])

In [ ]:
#cluster_feature = pd.DataFrame(cluster_feature)
#cluster_feature['campaign_id'] = campaign['campaign_id']

In [ ]:
#train_input['campaign_id'] = train['campaign_id']
#train_input = train_input.merge(cluster_feature,on = 'campaign_id',how = 'left')

In [ ]:
test_input = test_input.merge(user_features_matrix,on = 'user_id',how = 'left')
#test_input['campaign_id'] = test['campaign_id']
#test_input = test_input.merge(cluster_feature,on = 'campaign_id',how = 'left')

In [ ]:
#x = encoded_features(train_input,train_input)
#y = encoded_features(df = train_input,df_new=test_input)
x = train_input.copy()
y = test_input.copy()
count_feature = pd.concat([train['user_id'],test['user_id']])
count_feature = count_feature.value_counts()
x['counts'] = x['user_id'].map(count_feature)
y['counts'] = y['user_id'].map(count_feature)
x = x.drop(['user_id','is_click','is_open','id','campaign_id','communication_type'],axis = 1)
y = y.drop(['user_id','id','campaign_id','communication_type'],axis = 1)
#x['total_open_percentage'] = x['total_open']/x['counts']
#y['total_open_percentage'] = y['total_open']/y['counts']

In [ ]:
#x.drop('Unnamed: 0',axis = 1,inplace=True)
#y.drop('Unnamed: 0',axis = 1,inplace=True)
print(x.shape)
print(y.shape)

In [ ]:
print(train_input.shape)
print(test_input.shape)

In [ ]:
### Check the columns in X and Y
### They should be same and id variables should not be present
print(x.columns)
print('*-'*50)
print(y.columns)

In [ ]:
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression

xg = XGBClassifier(n_estimators = 600,max_depth = 6,gamma = 10)
#cb = CatBoostClassifier()
#knn = KNeighborsClassifier()
#rf = RandomForestClassifier()
#et = ExtraTreesClassifier()
#lr = LogisticRegression()

In [ ]:
### Choose your algo and use fit method (replace classifier by the name of your algo constructor)
xg.fit(x,train['is_click'])


In [ ]:
predictions = xg.predict_proba(y)

In [ ]:
sample = pd.read_csv(path + 'sample.csv')

In [ ]:
name_of_submission_file = 'final_sub'    # select name of file
sample['is_click'] = predictions[:,1]
sample.to_csv(path + '{}.csv'.format(name_of_submission_file), index=False)